Angels plants stock system, By Andrew Redford. Date: 5 September 2023

set up imports required

In [ ]:
from datetime import datetime
import IPython.display
import os.path
import pandas as pd
import csv
import numpy as np
import pprint

ensure user and password are set up so pandas functions with postgresql databases

In [ ]:
DB_USER='tm351'
DB_PWD='tm351'

In [ ]:
# Make the connection
%run sql_init.ipynb
print("Connecting with connection string : {}".format(DB_CONNECTION))
%sql $DB_CONNECTION

%run reset_databases.ipynb

In [ ]:

%%sql users_tables <<
    SELECT tablename
    FROM pg_tables
    WHERE tableowner=(SELECT current_user)
        AND schemaname='public'

In [ ]:
if not users_tables.empty:
    users_tables=list(users_tables['tablename'])
else:
    users_tables=[]

In [ ]:
tables_to_drop_ls=['Reserve_orders', 'Nursery_managers', 'Order_lines', 
                   'Sales_team_members', 'Plant_orders', 'Customers', 
                   'Customer_groups', 'Stock_splitting_records', 'Plants',
                   'reserve_orders', 'nursery_managers', 'order_lines', 
                   'sales_team_members', 'plant_orders', 'customers', 
                   'customer_groups', 'stock_splitting_records', 'plants']

In [ ]:
for table in tables_to_drop_ls:
    if table in users_tables:
        IPython.display.display_pretty('Removing table {}'.format(table))
        %sql DROP TABLE IF EXISTS $table CASCADE



reset schema in case it is full of data

In [ ]:
IPython.display.display_pretty('Reset angelsplants schema')

In [ ]:
%%sql

DROP SCHEMA IF EXISTS angelsplants CASCADE;
DROP SCHEMA IF EXISTS angelsplants Plants;

drop schema if exists

In [ ]:
%%sql

DROP TABLE IF EXISTS reserve_orders CASCADE;


now create the schema

In [ ]:

%%sql

CREATE SCHEMA ANGELSPLANTS;

create the tables needed for the ANGELSPLANTS database putting prompt to screen before each: dropping table first

CHANGED ORDER

In [ ]:
%%sql
--Table 1 of 9 
--drop if exists then create table Customer_groups
DROP TABLE IF EXISTS "Customer_groups" CASCADE;
DROP TABLE IF EXISTS customer_groups CASCADE;
CREATE TABLE Customer_groups ( 
    --PK
    group_id varchar (13) NOT NULL,
    --normal
    group_name varchar (30),
    group_address varchar (60),
    group_tel_no varchar (30));

In [ ]:
%%sql
--Table 2 of 9 
--drop if exists then create table Reserve_orders
DROP TABLE IF EXISTS "Reserve_orders" CASCADE;
DROP TABLE IF EXISTS reserve_orders CASCADE;
CREATE TABLE Reserve_orders ( 
    --PK
    reserve_order_id varchar (13) NOT NULL,
    --FK 
    customer_id varchar (13) NOT NULL,
    plant_id varchar (13) NOT NULL,
    manager_id varchar (13) NOT NULL,
    --normal
    reserve_quantity_allowed int,
    reserve_stock_remaining int);

In [ ]:
%%sql
--Table 3 of 9
--drop if exists then create table Nursery_managers
DROP TABLE IF EXISTS "Nursery_managers" CASCADE;
DROP TABLE IF EXISTS nursery_managers CASCADE;
CREATE TABLE Nursery_managers ( 
    --PK
    manager_id varchar (13) NOT NULL,
    --normal fields 
    first_name varchar (30),
    last_name varchar (30));

In [ ]:
%%sql
--Table 4 of 9 
--drop if exists then create table Order_lines
DROP TABLE IF EXISTS "Order_lines" CASCADE;
DROP TABLE IF EXISTS order_lines CASCADE;
CREATE TABLE Order_lines ( 
    --PK
    order_line_id varchar (13) NOT NULL,
    --FK 
    plant_order_id varchar (13) NOT NULL,
    plant_id varchar (13) NOT NULL,
    --normal
    stock_units_rqd int,
    total_units_to_remove_from_stock int,
    vat float (8),
    order_line_total float (8));

In [ ]:
%%sql
--Table 5 of 9
--drop if exists then create table Sales_team_members
DROP TABLE IF EXISTS "Sales_team_members" CASCADE;
DROP TABLE IF EXISTS sales_team_members CASCADE;
CREATE TABLE Sales_team_members ( 
    --PK
    sales_team_id varchar (13) NOT NULL,
    --normal fields 
    first_name varchar (30),
    last_name varchar (30));

In [ ]:
%%sql
--Table 6 of 9 
--drop if exists then create table Plant_orders
DROP TABLE IF EXISTS "Plant_orders" CASCADE;
DROP TABLE IF EXISTS plant_orders CASCADE;
CREATE TABLE Plant_orders ( 
    --PK
    plant_order_id varchar (13) NOT NULL,
    --FK 
    customer_id varchar (13) NOT NULL,
    sales_team_id varchar (13) NOT NULL,
    --normal
    ack_sent char,
    date_order_ack_sent date,
    grand_total float (8));

In [ ]:
%%sql
--Table 7 of 9 
--drop if exists then create table Customers
DROP TABLE IF EXISTS "Customers" CASCADE;
DROP TABLE IF EXISTS customers CASCADE;
CREATE TABLE Customers (  
    --PK
    customer_id varchar (13) NOT NULL,
    --FK 
    customer_group_id varchar (13) NOT NULL,
    --normal
    customer_first_name varchar (30),
    customer_last_name varchar (30),
    customer_address varchar (60),
    customer_tel_no varchar (30));

In [ ]:
%%sql
--Table 8 of 9 
--drop if exists then create table Stock_splitting_records
DROP TABLE IF EXISTS "Stock_splitting_records" CASCADE;
DROP TABLE IF EXISTS stock_splitting_records CASCADE;
CREATE TABLE Stock_splitting_records ( 
    --PK
    stock_splitting_line_id varchar(13) NOT NULL,
    --FK 
    group_id varchar (13) NOT NULL,
    plant_id varchar (13) NOT NULL,
    --normal
    week_no int,
    year int,
    qty_of_plant_allocated int);


In [ ]:
%%sql
--Table 9 of 9 
--drop if exists then create table Plants
DROP TABLE IF EXISTS "Plants" CASCADE;
DROP TABLE IF EXISTS plants CASCADE;
CREATE TABLE Plants (
    --PK
    plant_id varchar(13) NOT NULL,
    --normal
    plant_type varchar(50),
    best_seller_status_yn char,
    plant_name varchar(50),
    qty_in_stock int,
    plant_barcode varchar(13),
    plant_price float,
    plant_size varchar (5),
    sold_in_multiples_of int,
    pack_or_pot varchar (4),
    measurement varchar (13));


UNIQUE AND PK CONSTRAINTS:

In [ ]:
%%sql
--Table 1 of 9 add unique constraint
ALTER TABLE Reserve_orders
    ADD CONSTRAINT un_reserve_order1 UNIQUE (reserve_order_id);

-- Drop the existing constraint
ALTER TABLE Reserve_orders 
    DROP CONSTRAINT IF EXISTS pk_reserve_orders;
    
ALTER TABLE Reserve_orders
    ADD CONSTRAINT pk_reserve_orders 
        PRIMARY KEY (reserve_order_id);
    

In [ ]:
%%sql
--Table 2 of 9 add unique constraint and pk
ALTER TABLE Nursery_managers
    ADD CONSTRAINT un_nursery_managers1 UNIQUE (manager_id);

-- Drop the existing constraint
ALTER TABLE Nursery_managers 
    DROP CONSTRAINT IF EXISTS pk_nursery_managers;
    
ALTER TABLE Nursery_managers
    ADD CONSTRAINT pk_nursery_managers
        PRIMARY KEY (manager_id);


In [ ]:
%%sql
--Table 3 of 9 add unique constraint and pk
ALTER TABLE Order_lines
    ADD CONSTRAINT un_order_lines1 UNIQUE (order_line_id);

-- Drop the existing constraint
ALTER TABLE Order_lines 
    DROP CONSTRAINT IF EXISTS pk_order_lines;
    
ALTER TABLE Order_lines
    ADD CONSTRAINT pk_order_lines
        PRIMARY KEY (order_line_id);

In [ ]:
%%sql
--Table 4 of 9 add unique constraint and pk
ALTER TABLE Sales_team_members
    ADD CONSTRAINT un_sales_team_members1 UNIQUE (sales_team_id);

-- Drop the existing constraint
ALTER TABLE Sales_team_members 
    DROP CONSTRAINT IF EXISTS pk_sales_team_members;
    
ALTER TABLE Sales_team_members
 ADD CONSTRAINT pk_sales_team_members
    PRIMARY KEY (sales_team_id);

In [ ]:
%%sql
--Table 5 of 9 add unique constraint and pk
ALTER TABLE Plant_orders
    ADD CONSTRAINT un_plant_orders1 UNIQUE (plant_order_id);

-- Drop the existing constraint
ALTER TABLE Plant_orders 
    DROP CONSTRAINT IF EXISTS pk_plant_orders;
    
ALTER TABLE Plant_orders
    ADD CONSTRAINT pk_plant_orders
        PRIMARY KEY (plant_order_id);


In [ ]:
%%sql
--Table 6 of 9 add unique constraint and pk
ALTER TABLE Customers
    ADD CONSTRAINT un_customers1 UNIQUE (customer_id);

-- Drop the existing constraint
ALTER TABLE Customers 
    DROP CONSTRAINT IF EXISTS pk_customers;

ALTER TABLE Customers
    ADD CONSTRAINT pk_customers
        PRIMARY KEY (customer_id);

In [ ]:
%%sql
--Table 7 of 9 add unique constraint and pk
ALTER TABLE Customer_groups
    ADD CONSTRAINT un_customer_groups1 UNIQUE (group_id);

-- Drop the existing constraint
ALTER TABLE Customer_groups 
    DROP CONSTRAINT IF EXISTS pk_customer_groups;

--add the constraint
ALTER TABLE Customer_groups
    ADD CONSTRAINT pk_customer_groups
        PRIMARY KEY (group_id);


In [ ]:
%%sql
--Table 8 of 9 add unique constraint and pk
ALTER TABLE Stock_splitting_records
    ADD CONSTRAINT un_stock_splitting_records1 UNIQUE (stock_splitting_line_id);

-- Drop the existing constraint
ALTER TABLE Stock_splitting_records 
    DROP CONSTRAINT IF EXISTS pk_stock_splitting_records;

--add the constraint
ALTER TABLE Stock_splitting_records
    ADD CONSTRAINT pk_stock_splitting_records
        PRIMARY KEY (stock_splitting_line_id);

In [ ]:
%%sql
--Table 9 of 9 add unique constraint and pk
ALTER TABLE Plants
    ADD CONSTRAINT un_plants1 UNIQUE (plant_id);

-- Drop the existing constraint
ALTER TABLE Plants 
    DROP CONSTRAINT IF EXISTS pk_plants;

--add the constraint
ALTER TABLE Plants
    ADD CONSTRAINT pk_plants
        PRIMARY KEY (plant_id);

FK Constraints:

In [ ]:
%%sql
--Table 1 of 9 add fk constraint
    
--add fk1
-- Drop the existing constraint
ALTER TABLE Reserve_orders 
DROP CONSTRAINT IF EXISTS fk_reserve_orders1;
--add the constraint
ALTER TABLE Reserve_orders
    ADD CONSTRAINT fk_reserve_orders1 
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id);
    
--add fk2
-- Drop the existing constraint
ALTER TABLE Reserve_orders 
DROP CONSTRAINT IF EXISTS fk_reserve_orders2;
--add the constraint
ALTER TABLE Reserve_orders
    ADD CONSTRAINT fk_reserve_orders2 
    FOREIGN KEY (plant_id) REFERENCES Plants(plant_id);
 
--add fk3
-- Drop the existing constraint
ALTER TABLE Reserve_orders 
DROP CONSTRAINT IF EXISTS fk_reserve_orders3;
--add the constraint
ALTER TABLE Reserve_orders
    ADD CONSTRAINT fk_reserve_orders3 
    FOREIGN KEY (manager_id) REFERENCES Nursery_managers(manager_id);
 

In [ ]:
#Table 2 of 9 add fk constraint
#there is no foreign key in this table Nursery managers


In [ ]:
%%sql
--Table 3 of 9 add fk constraints

--add fk1
-- Drop the existing constraint
ALTER TABLE Order_lines 
DROP CONSTRAINT IF EXISTS fk_order_lines1;
--add the constraint
ALTER TABLE Order_lines
    ADD CONSTRAINT fk_order_lines1 
    FOREIGN KEY (plant_order_id) REFERENCES Plant_orders(plant_order_id);
    
--add fk2
-- Drop the existing constraint
ALTER TABLE Order_lines 
DROP CONSTRAINT IF EXISTS fk_order_lines2;
--add the constraint
ALTER TABLE Order_lines
    ADD CONSTRAINT fk_order_lines2 
    FOREIGN KEY (plant_id) REFERENCES Plants(plant_id);

    

In [ ]:
#Table 4 of 9 add unique constraint
#there is no foreign key in table Sales_team_members


In [ ]:
%%sql
--Table 5 of 9 add fk constraints

--add fk1
-- Drop the existing constraint
ALTER TABLE Plant_orders 
DROP CONSTRAINT IF EXISTS fk_plant_orders1;
--add the constraint
ALTER TABLE Plant_orders
    ADD CONSTRAINT fk_plant_orders1 
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id);
    
--add fk2
-- Drop the existing constraint
ALTER TABLE Plant_orders 
DROP CONSTRAINT IF EXISTS fk_plant_orders2;
--add the constraint
ALTER TABLE Plant_orders
    ADD CONSTRAINT fk_plant_orders2 
    FOREIGN KEY (sales_team_id) REFERENCES Sales_team_members(sales_team_id);

In [ ]:
%%sql
--Table 6 of 9 add fk constraint

--add fk1
-- Drop the existing constraint
ALTER TABLE Customers 
DROP CONSTRAINT IF EXISTS fk_customers1;
--add the constraint
ALTER TABLE Customers
    ADD CONSTRAINT fk_customers1 
    FOREIGN KEY (customer_group_id) REFERENCES Customer_groups(group_id);
    

In [ ]:
#Table 7 of 9 no fk constraints for customer groups


In [ ]:
%%sql
--Table 8 of 9 add fk constraint

--add fk1
-- Drop the existing constraint
ALTER TABLE Stock_splitting_records 
DROP CONSTRAINT IF EXISTS fk_stock_splitting_records1;
--add the constraint
ALTER TABLE Stock_splitting_records
    ADD CONSTRAINT fk_stock_splitting_records1 
    FOREIGN KEY (group_id) REFERENCES Customer_groups(group_id);

--add fk2
-- Drop the existing constraint
ALTER TABLE Stock_splitting_records 
DROP CONSTRAINT IF EXISTS fk_stock_splitting_records2;
--add the constraint
ALTER TABLE Stock_splitting_records
    ADD CONSTRAINT fk_stock_splitting_records2 
    FOREIGN KEY (plant_id) REFERENCES Plants(plant_id);
    
    
    

In [ ]:
#Table 9 of 9 add fk constraint
#Table Plants has no foreign keys     

In [ ]:
%schema --connection_string $DB_CONNECTION -t Customer_groups 

In [ ]:
%schema --connection_string $DB_CONNECTION -t Reserve_orders 

In [ ]:
%schema --connection_string $DB_CONNECTION -t Nursery_managers

In [ ]:
%schema --connection_string $DB_CONNECTION -t Order_lines

In [ ]:
%schema --connection_string $DB_CONNECTION -t Sales_team_members

In [ ]:
%schema --connection_string $DB_CONNECTION -t Plant_orders

In [ ]:
%schema --connection_string $DB_CONNECTION -t Customers

In [ ]:
%schema --connection_string $DB_CONNECTION -t Customer_groups

In [ ]:
%schema --connection_string $DB_CONNECTION -t Stock_splitting_records

In [ ]:
%schema --connection_string $DB_CONNECTION -t Plants

In [ ]:
%schema --connection_string $DB_CONNECTION -t ,plant_orders,order_lines,nursery_managers,reserve_orders,plants,stock_splitting_records,customer_groups,customers

Important: Enable Cell below on 2nd run and thereafter. Disable it and enable the one below for first run

#This cell only after first run: It makes dictionaries from csvs of existing test data.
#To make test data exist: disable this cell and enable the 2 code cells after it.
    
#get dataframe from csv file and start with index of 1 not 0
df1 = pd.read_csv('sql_data/Customer_groups_members_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Customer_groups_members_dict = df1.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df2 = pd.read_csv('sql_data/Reserve_orders_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Reserve_orders_dict = df2.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df3 = pd.read_csv('sql_data/Nursery_managers_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Nursery_managers_dict = df3.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df4 = pd.read_csv('sql_data/Order_lines_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Order_lines_dict = df4.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df5 = pd.read_csv('sql_data/Sales_team_members_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Sales_team_members_dict = df5.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df6 = pd.read_csv('sql_data/Plant_orders_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Plant_orders_dict = df6.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df7 = pd.read_csv('sql_data/Customers_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Customers_dict = df7.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df8 = pd.read_csv('sql_data/Stock_splitting_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Stock_splitting_dict = df8.to_dict()

#get dataframe from csv file and start with index of 1 not 0
df9 = pd.read_csv('sql_data/Plants_dict.csv',header=[0], index_col=[0]).T
#get dict from dataframe of csv file
Plants_dict = df9.to_dict()


#stores the information for reserve orders and stock splitting and plant stock after 
#order allocation and changes have rippled down through all dictionaries. 
#This is up to date changes
updated_everything_dict = {}

#these store plant id stock units required to a dictionary for adding to a dataframe later  
#this is info that is not saved to the main dictionaries but needed to display after 
#changes have been made
plant_id_allocated={}
plant_id_stock={}

For first time program is run, ensure you run next 2 cells, instead of above one:

In [ ]:
### Make dictionaries used by main. 
#Enable this cell on first run to create csv test data, disable thereafter.


#about allocation of plants to customer groups
#stock_splitting_line_id is the key
Stock_splitting_dict = {1: {'group_id' : 1, #independent
                            'week_no' : 36,
                            'year' : '2023', 
                            'plant_id' : 1,  #13 digits are used usually, for test ok 1 digit
                            'qty_of_plant_allocated' : 990},

                        2: {'group_id' : 2, #sunnyridge
                            'week_no' : 36,
                            'year' : '2023', 
                            'plant_id' : 1, 
                            'qty_of_plant_allocated' : 0},

                        3: {'group_id' : 3, #greenheart
                            'week_no' : 36,
                            'year' : '2023', 
                            'plant_id' : 2, 
                            'qty_of_plant_allocated' : 0},

                        4: {'group_id' : 1, 
                            'week_no' : 37,
                            'year' : '2023', 
                            'plant_id' : 2, #this and next 2 are wk 2 2023 same plant id
                            'qty_of_plant_allocated' : 15},

                        5: {'group_id' : 2, 
                            'week_no' : 37,
                            'year' : '2023', 
                            'plant_id' : 2, 
                            'qty_of_plant_allocated' : 25},

                        6: {'group_id' : 3, 
                            'week_no' : 37,
                            'year' : '2023', 
                            'plant_id' : 2, 
                            'qty_of_plant_allocated' : 35}}

#final version the group id must not be allowed to be more than the number of groups
#that exist so this would not be allowed as we only have 3 groups in the system:
                        #7: {'group_id' : 4,  #this and next 2 are week 3 2023 for ...02 plant id
                        #    'week_no' : 38,
                        #    'year' : '2023', 
                        #    'plant_id' : 3, 
                        #    'qty_of_plant_allocated' : 35}}


#plant_id is the key
Plants_dict = { 1: {'plant_type' : 'Bedding',  #1 is the key which is sunnyridge
                    'best_seller_status_yn' : 'y',
                    'plant_name' : 'Pansy White', #can be same as another in dictionary but must have different size
                    'qty_in_stock' : 990, #because it is a multiple of 15
                    'plant_barcode' : '7231231312391', #always 13 digits
                    'plant_price' : 0.8, #typical real world cost price                    
                    'plant_size' : '10.5', 
                    'sold_in_multiples_of' : 15, #it is sold in trays of 15
                    'pack_or_pot' : 'pot',
                    'measurement' : 'cm' },


#                2: {'plant_type' : 'Bedding', #greenheart is the second key
#                    'best_seller_status_yn' : 'y',
#                    'plant_name' : 'Pansy White', 
#                    'qty_in_stock' : 100,
#                    'plant_barcode' : '9231231231341',
#                    'plant_price' : 1.4,                    
#                    'plant_size' : '6.0', #because it is a pack this refers to 6 pack 
#                    'sold_in_multiples_of' : 1, #must be one as sold in a pack
#                    'pack_or_pot' : 'pack',
#                    'measurement' : 'plantcell'},

                3: {'plant_type' : 'Bedding', #3 is the key which is independents
                    'best_seller_status_yn' : 'y',
                    'plant_name' : 'Geranium Mix', 
                    'qty_in_stock' : 0,# 100,
                    'plant_barcode' : '2211111111123',
                    'plant_price' : 3.5,                    
                    'plant_size' : '20.0', #because it is a pack this refers to 20 pack 
                    'sold_in_multiples_of' : 1, #must be one as sold in a pack
                    'pack_or_pot' : 'pack',
                    'measurement' : 'plantcell'},

                4: {'plant_type' : 'Bedding',
                    'best_seller_status_yn' : 'n',
                    'plant_name' : 'Marigold French Yellow', 
                    'qty_in_stock' : 900,# 100,
                    'plant_barcode' : '1211111111124',
                    'plant_price' : 0.75,                    
                    'plant_size' : '9.0', 
                    'sold_in_multiples_of' : 18, #because it is a 9cm
                    'pack_or_pot' : 'pot',
                    'measurement' : 'cm'},

                5: {'plant_type' : 'Herbaceous',
                    'best_seller_status_yn' : 'n',
                    'plant_name' : 'Dahlia Coral Flame', 
                    'qty_in_stock' : 100,# 100, 
                    'plant_barcode' : '4211111111124',
                    'plant_price' : 2.3,                    
                    'plant_size' : '2.0', 
                    'sold_in_multiples_of' : 1, #because it is a 2 litre
                    'pack_or_pot' : 'pot',
                    'measurement' : 'litre'},

                6: {'plant_type' : 'Herbaceous',
                    'best_seller_status_yn' : 'n',
                    'plant_name' : 'Salvia Blue', 
                    'qty_in_stock' : 0,# 100,
                    'plant_barcode' : '2321111801124',
                    'plant_price' : 3.9,                    
                    'plant_size' : '5.0', 
                    'sold_in_multiples_of' : 1, #because it is a 5 litre
                    'pack_or_pot' : 'pot',
                    'measurement' : 'litre'}}



#group_id is the key
Customer_groups_members_dict = { 1 : {'group_name': 'Sunnyridge', 
                                      'group_address': 'Sunnyridge HQ, Shepherds Bush, London.', 
                                      'group_tel_no': 'Huckstable'},

                                 2 : {'group_name': 'Greenheart', 
                                      'group_address': 'Greenheart  HQ, Camden Town, London', 
                                      'group_tel_no': 'Huckstable'},

                                 3 : {'group_name': 'Independents', 
                                      'group_address': 'Various Addresses', 
                                      'group_tel_no': 'Various nums'}}

    
#reserve_order_id is the key
Reserve_orders_dict =  { 1: {'customer_id' : 1, #From Sunnyridge group, Sunnyridge Branch 5
                             'plant_id' : 1,
                             'manager_id' : 1, #rudolph
                             'reserve_quantity_allowed' : 210, #must be in multiples of 15
                             'reserve_stock_remaining' : 210}, #must be in multiples of 15

                         2: {'customer_id' : 1, #from Greenheart group, Greenheart Branch 4
                             'plant_id' : 2,   #can be in multiples of 1
                             'manager_id' : 1, #rudolph 
                             'reserve_quantity_allowed' : 200,
                             'reserve_stock_remaining' : 194},

                         3: {'customer_id' : 2, #from Greenheart group, Greenheart Branch 4
                             'plant_id' : 1,
                             'manager_id' : 1, #rudolph
                             'reserve_quantity_allowed' : 195,
                             'reserve_stock_remaining' : 195},

                         4: {'customer_id' : 2, #from Greenheart group, Greenheart Branch 4
                             'plant_id' : 2,
                             'manager_id' : 1, #rudolph
                             'reserve_quantity_allowed' : 300,
                             'reserve_stock_remaining' : 200},

                         5: {'customer_id' : 3, #from Greenheart group, Greenheart Branch 4
                             'plant_id' : 1,
                             'manager_id' : 1, #rudolph
                             'reserve_quantity_allowed' : 45,
                             'reserve_stock_remaining' : 45},

                         6: {'customer_id' : 3, #from Greenheart group, Greenheart Branch 4
                             'plant_id' : 2,
                             'manager_id' : 1, #rudolph
                             'reserve_quantity_allowed' : 50,
                             'reserve_stock_remaining' : 10}}


#for testing. I Put two into each group  
#customer_id is the key
Customers_dict = { 1 : {'customer_group_id' : 1, #sunnyridge
                        'customer_first_name' : 'Simon',
                        'customer_last_name' : 'Sunday',
                        'customer_address' : 'Sunnyridge Branch 5, Sunnyridge lane, London, UK',
                        'customer_tel_no' : '0900 232 232'},

                     2: {'customer_group_id' : 1, #sunnyridge 
                        'customer_first_name' : 'Susan',
                        'customer_last_name' : 'Sussex',
                        'customer_address' : 'Sunnyridge Branch 2, Sunnyridge lane, London, UK',
                        'customer_tel_no' : '02300 442 241'},

 #                    3: {'customer_group_id' : 2, #Greenheart
 #                       'customer_first_name' : 'Joy',
 #                       'customer_last_name' : 'Grey',
 #                       'customer_address' : 'Greenheart Branch 4, Allgood lane, Hertforshire',
 #                       'customer_tel_no' : '03304 243 222'},

                     4: {'customer_group_id' : 2, #Greenheart
                        'customer_first_name' : 'Tate',
                        'customer_last_name' : 'Greenway',
                        'customer_address' : 'Greenheart Branch 1, Woking, Surrey',
                        'customer_tel_no' : '01180 431 111'},

                     5: {'customer_group_id' : 3, #Independents
                        'customer_first_name' : 'Tom',
                        'customer_last_name' : 'Innes',
                        'customer_address' : 'Innes garden centre, Stroud, Gloucestershire',
                        'customer_tel_no' : '01453 479 238'},

                     6: {'customer_group_id' : 3, #Independents
                        'customer_first_name' : 'Bob',
                        'customer_last_name' : 'India',
                        'customer_address' : 'India garden centre, Slade, Reading',
                        'customer_tel_no' : '01179 439 238'}}


#manager_id is the key
Nursery_managers_dict = { 1 : {'first_name': 'Rudolph', 'last_name': 'Huckstable'},
                          2 : {'first_name': 'Theresa', 'last_name': 'March'}}


Plant_orders_dict = {1: {'customer_id' : 1, 
                         'sales_team_id': 1, #Andrew
                         'ack_sent': 'y',
                         'date_order_ack_sent': '2019-01-04',
                         'grand_total' : 2.3},

                     2: {'customer_id' : 2, 
                         'sales_team_id': 1, #Andrew
                         'ack_sent': 'y',
                         'date_order_ack_sent': '2019-01-04',
                         'grand_total' : 2.4},

                     3: {'customer_id' : 3, 
                         'sales_team_id': 2, #Tom
                         'ack_sent': 'n',
                         'date_order_ack_sent': '2019-01-04',
                         'grand_total' : 2.5}, 

                     4: {'customer_id' : 1, 
                         'sales_team_id': 2, #Tom
                         'ack_sent': 'y',
                         'date_order_ack_sent': '2019-01-05',
                         'grand_total' : 2.6},

                     5: {'customer_id' : 2, 
                         'sales_team_id': 3, #Jane
                         'ack_sent': 'n',
                         'date_order_ack_sent': '2019-01-05',
                         'grand_total' : 2.7},

                     6: {'customer_id' : 3, 
                         'sales_team_id': 3, #Jane
                         'ack_sent': 'n',
                         'date_order_ack_sent': '2020-01-05',
                         'grand_total' : 2.8}}
#how to put todays date into the above dictionary:
date=datetime.today().strftime('%Y-%m-%d')
Plant_orders_dict[1]['date_order_ack_sent'] = date
#print(Plant_orders_dict[1]['date_order_ack_sent'])


#each order had 2 lines in it. 
#order_line_id is the key
#testing:for each order I put 2 plant IDs in. same in each order.
Order_lines_dict = {1: {'plant_order_id' : 1, #can be same plant_order_id in multiple
                        'plant_id' : 1, #if same plant order id as in another key,plant id will be different for each key
                        'stock_units_rqd' : 2,
                        'total_stock_to_take_out': 1,
                        'vat': 0.2,
                        'order_line_total': 2},

                    2: {'plant_order_id' : 1,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 3},

                    3: {'plant_order_id' : 2,
                        'plant_id' : 1,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 4},

                    4: {'plant_order_id' : 2,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 5},

                    5: {'plant_order_id' : 3,
                        'plant_id' : 1,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 6},

                    6: {'plant_order_id' : 3,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 7},

                    7: {'plant_order_id' : 4,
                        'plant_id' : 1,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 8},

                    8: {'plant_order_id' : 4,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 9},

                    9: {'plant_order_id' : 5,
                        'plant_id' : 1,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 10},

                    10: {'plant_order_id' : 5,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 11},

                    11: {'plant_order_id' : 6,
                        'plant_id' : 1,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 12},

                    12: {'plant_order_id' : 6,
                        'plant_id' : 2,
                        'stock_units_rqd' : 1,
                        'total_stock_to_take_out': 18,
                        'vat': 1.2,
                        'order_line_total': 13}}


Sales_team_members_dict = { 1 : {'first_name': 'Andrew', 'last_name': 'Harris'},
                            2 : {'first_name': 'Tom', 'last_name': 'Barnes'},
                            3 : {'first_name': 'Jane', 'last_name': 'Long'}}

#stores the information for reserve orders and stock splitting and plant stock after 
#order allocation and changes have rippled down through all dictionaries. 
#This is up to date changes
updated_everything_dict = {}

#these store plant id stock units required to a dictionary for adding to a dataframe later  
#this is info that is not saved to the main dictionaries but needed to display after 
#changes have been made
plant_id_allocated={}
plant_id_stock={}



#this step makes dataframes from the dictionaries,needed for creating test data
#Use the first time you run the program only

#df1 = df.reset_index()
df1 = pd.DataFrame(Customer_groups_members_dict).T
#df1.columns = ['group_name','group_address','group_tel_no']

#makes is look good in csv:
df1.to_csv('sql_data/Customer_groups_members_dict.csv', index = 1, index_label='Group id')
#df1.to_csv('sql_data/Customer_groups_members_dict.csv', index = False)

#df2 = df.reset_index()
df2 = pd.DataFrame(Reserve_orders_dict).T
df2.to_csv('sql_data/Reserve_orders_dict.csv',index = 1, index_label='Reserve order id')
Customer_groups_members_dict = df1.to_dict('index')

#df3 = df.reset_index()
df3 = pd.DataFrame(Nursery_managers_dict).T
df3.to_csv('sql_data/Nursery_managers_dict.csv', index = 1, index_label='Manager id')

#df4 = df.reset_index()
df4 = pd.DataFrame(Order_lines_dict).T
df4.to_csv('sql_data/Order_lines_dict.csv', index = 1, index_label='Order line id')

#df5 = df.reset_index()
df5 = pd.DataFrame(Sales_team_members_dict).T
df5.to_csv('sql_data/Sales_team_members_dict.csv', index = 1, index_label='Sales team id')

#df6 = df.reset_index()
df6 = pd.DataFrame(Plant_orders_dict).T
#df6.to_csv('sql_data/Plant_orders_dict.csv', index = False)
df6.to_csv('sql_data/Plant_orders_dict.csv', index = 1, index_label='Plant order id')

#df7 = df.reset_index()
df7 = pd.DataFrame(Customers_dict).T
df7.to_csv('sql_data/Customers_dict.csv', index = 1, index_label='Customer id')

#df8 = df.reset_index()
df8 = pd.DataFrame(Stock_splitting_dict).T
df8.to_csv('sql_data/Stock_splitting_dict.csv', index = 1, index_label='Stock line id')

#df9 = df.reset_index()
df9 = pd.DataFrame(Plants_dict).T
df9.to_csv('sql_data/Plants_dict.csv', index = 1, index_label='Plant id')



this step makes sql tables from dataframes. New sql tables are:
Customer_groups,  Reserve_orders,  Nursery_managers, Order_lines,  Sales_team_members, Plant_orders, 
Customers, Stock_splitting_records, Plants 

In [ ]:

df1.to_sql('Customer_groups',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df2.to_sql('Reserve_orders',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df3.to_sql('Nursery_managers',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df4.to_sql('Order_lines',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df5.to_sql('Sales_team_members',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df6.to_sql('Plant_orders',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df7.to_sql('Customers',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df8.to_sql('Stock_splitting_records',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)
df9.to_sql('Plants',
                    DB_CONNECTION,
                    if_exists='replace',
                    index=False)


the functions to be called from main:

In [58]:
def add_ord_pt4_update_rsv_ord_stock_lvls(plant_id, customer_id, total_units_to_remove_from_stock):
    #Problem 2.2
    reserve_quantity_allowed=0
    new_reserve_stock_remaining=0
    reserve_stock_taken_so_far=0
    for reserve_order_id in Reserve_orders_dict:
        if (int(Reserve_orders_dict[reserve_order_id]['plant_id']) == plant_id and
                int(Reserve_orders_dict[reserve_order_id]['customer_id']) == customer_id):
            reserve_quantity_allowed=int(Reserve_orders_dict[reserve_order_id]['reserve_quantity_allowed'])
            #do the maths and save to a variable
            old_reserve_stock_remaining=int(Reserve_orders_dict[reserve_order_id]['reserve_stock_remaining'])
            new_reserve_stock_remaining=int(old_reserve_stock_remaining-total_units_to_remove_from_stock)
            reserve_stock_taken_so_far=reserve_quantity_allowed-new_reserve_stock_remaining
            Reserve_orders_dict[reserve_order_id]['reserve_stock_remaining'] = \
            str(new_reserve_stock_remaining)
    return(reserve_quantity_allowed,new_reserve_stock_remaining,reserve_stock_taken_so_far)


def add_ord_pt3_update_stock_splitting_stock_lvls (curr_week,curr_year, plant_id,total_units_to_remove_from_stock, group_id_to_update):
    #As part of problem 1.2
    #Reduce stock in Stock Splitting table by that amount for current group:
    #For any one plant id, there will be only one independent_split for this week and year, 
    #one sunnyridge_split and one for greenheart_split 
    independent_split=0
    sunnyridge_split=0
    greenheart_split=0
    for stock_splitting_line_id in Stock_splitting_dict:
        #print('----------------------')
        #print('CURRENTLY LOOKING AT ID ', stock_splitting_line_id, ' IN STOCK SPLITTING DICT')
        if (int(Stock_splitting_dict[stock_splitting_line_id]['plant_id']) == plant_id 
                and int(Stock_splitting_dict[stock_splitting_line_id]['week_no']) == curr_week
                and int(Stock_splitting_dict[stock_splitting_line_id]['year']) == curr_year
                and int(Stock_splitting_dict[stock_splitting_line_id]['group_id'])  == group_id_to_update):
            
            old_value_of_qty_of_plant_allocated=int(Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated'])
            Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated'] = str(old_value_of_qty_of_plant_allocated - total_units_to_remove_from_stock)
            #for loose coupling, to keep classes independent do it all in the class’s function
            new_value_for_this_group=Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated']

            if group_id_to_update == 1:
                print ('updating sunnyridge split group')
                sunnyridge_split = new_value_for_this_group
            elif group_id_to_update == 2:
                print ('updating greenheart split group')
                greenheart_split = new_value_for_this_group
            elif group_id_to_update == 3:
                print ('updating independent split group')
                independent_split = new_value_for_this_group
            else:
                print ('ERROR found no groups to update in stock splitting')
    #update stock splitting stock levels
    return(independent_split,sunnyridge_split, greenheart_split)
      


def add_ord_pt_2_update_plant_id_stock_lvls(plant_id_user_choice,total_units_to_remove_from_stock):
    #As part of problem 1,this is 1.2:
    #updates the stock level for one plant id 
    Plants_dict[plant_id_user_choice]['qty_in_stock'] = str(int(Plants_dict[plant_id_user_choice]['qty_in_stock']) - total_units_to_remove_from_stock)
    new_plant_id_stock=Plants_dict[plant_id_user_choice]['qty_in_stock']
    return(new_plant_id_stock)

    
def add_ord_pt1_enter_quant_calc_totals(this_order_id, plant_id_user_choice,stock_line_number,grand_total,dframe,\
                                        set_of_failed_updates):
    #get plant quantity required
    #copy dataframe to use to show what values would be without changing them in database
    
    #save to variables from dictionary for readability
    curr_plant_name=Plants_dict[plant_id_user_choice]['plant_name']
    curr_plant_size=Plants_dict[plant_id_user_choice]['plant_size']
    curr_plant_measurement=Plants_dict[plant_id_user_choice]['measurement']
    curr_plant_pack_or_pot=Plants_dict[plant_id_user_choice]['pack_or_pot']
    curr_qty_in_stock =int(Plants_dict[plant_id_user_choice]['qty_in_stock'])
    plant_price=float(Plants_dict[plant_id_user_choice]['plant_price'])
    plant_barcode=Plants_dict[plant_id_user_choice]['plant_barcode']    
    stock_units_rqd=int(input('Enter quantity required:'))
    
    #set the relevant  row with  value just entered for stock required 
    dataframe_row=plant_id_user_choice-1
    dframe.at[dataframe_row,'stock units rqd']=stock_units_rqd
    plant_id_stock[int(plant_id_user_choice)] = stock_units_rqd
    sold_in_multiples_of=int(Plants_dict[plant_id_user_choice]['sold_in_multiples_of'])
    theoretical_total_units_to_remove_from_stock = (stock_units_rqd * sold_in_multiples_of)
    theoretical_new_stock_value = int(curr_qty_in_stock) - int(theoretical_total_units_to_remove_from_stock)
    
    #make copies so we can work on them in the if statement, will help displaying later outside this function
    dframe_plants_in_stock=dframe
    dframe_all_plants=dframe
    
    if theoretical_new_stock_value < 0:
        #if quantity of  this plant id user wants is not in stock 
        #add it to a set storing set of failed updates 
        set_of_failed_updates.add(plant_id_user_choice)
        
        #column for this plant will go from Not attempted to Yes
        dframe_plants_in_stock.at[plant_id_user_choice-1,'allocated']='No'
        plant_id_allocated[int(plant_id_user_choice)]='No'
        
        #as there was no stock to do the updates we set these to zero to be passed back
        vat_for_one_plant=0
        order_line_total=0
        total_units_to_remove_from_stock=0
            
    else:
        #quantity of  this plant id user wants is in stock 

        if plant_id_user_choice in set_of_failed_updates:
            #if quantity of  this plant id user wants was once entered as a too-high-quantity (meaning that plant id 
            #was have saved in a set as attempted but failed, and shown as a red row to the user),
            #remove the plant id from that set of failed updates as it can not be updated succesfully, as user has .
            #entered a quantity that is in stock
            set_of_failed_updates.remove(plant_id_user_choice)
        
        dframe_all_plants.at[plant_id_user_choice-1,'allocated']='Yes'
        plant_id_allocated[int(plant_id_user_choice)]='Yes'
        
        #make the newest order line id (the primary key for order lines) to be the last one plus one
        order_line_id = generate_unique_id(Order_lines_dict)
        #print('new order_line_id=',order_line_id)
        
        #current order lines dictionary has unique identifiers for primary keys, now for each 
        #we need to setup another dictionary for secondary keys
        Order_lines_dict[order_line_id]={}
        
        #not primary keys in order_lines_dict so treated as strings:
        #these are the foreign aka secondary keys 
        Order_lines_dict[order_line_id]['plant_order_id']=int(this_order_id)
        Order_lines_dict[order_line_id]['plant_id']=int(plant_id_user_choice)
        Order_lines_dict[order_line_id]['stock_units_rqd']=int(stock_units_rqd)
        #going to do mathematical operation with this so needs to be int for that
        #sold_in_multiples_of=int(Plants_dict[plant_id_user_choice]['sold_in_multiples_of'])
        #get this to send back from function though not used in this funciton will be used in main
        #plant_barcode=Plants_dict[plant_id_user_choice]['plant_barcode']
        #work out total units to remove from stock
        total_units_to_remove_from_stock = (stock_units_rqd * sold_in_multiples_of)


        #save back to dictionary as a string after operation done on it
        Order_lines_dict[order_line_id]['total_stock_to_take_out']=\
        int(total_units_to_remove_from_stock)
        #vat for one plant gives over 10 decimal places 
        vat_for_one_plant=plant_price * 0.2
        #rounded to get rid of too many decimal places
        vat_for_one_plant = round(vat_for_one_plant, 2)

        #save back to dictionary as a string after operation done on it
        Order_lines_dict[order_line_id]['vat']=vat_for_one_plant
        #print('vat_for_one_plant:',vat_for_one_plant)
        #print('total_units_to_remove_from_stock*vat_for_one_plant:',total_units_to_remove_from_stock*vat_for_one_plant)
        #print('total_units_to_remove_from_stock*plant_price:',total_units_to_remove_from_stock*plant_price)
        #all int values below
        order_line_total=(total_units_to_remove_from_stock*vat_for_one_plant)+(total_units_to_remove_from_stock*plant_price)
        #print('order_line_total(those two figues added)',order_line_total)
        #save back to dictionary as a string after operation done on it
        Order_lines_dict[order_line_id]['order_line_total']=order_line_total
        #add the line total to the grand total
        grand_total=grand_total+order_line_total
        #increase stock line number by one and return it
        
        #we only increase this here if it was a succesful update and there was stock to do the calculations
        stock_line_number += 1
    return(curr_plant_name, curr_plant_size, curr_plant_measurement, curr_plant_pack_or_pot, \
           curr_qty_in_stock, plant_barcode,plant_price, vat_for_one_plant, order_line_total,\
           sold_in_multiples_of, stock_units_rqd, \
           grand_total,stock_line_number,total_units_to_remove_from_stock,set_of_failed_updates,\
           theoretical_new_stock_value,dframe_all_plants)

        
class Plants:    
    def display_plant_info(self,user_choice, customer_id_input, curr_week, curr_year,stock_units_rqd):
        plant_stock_data = []
        print('')
        if user_choice == 1:
            
            print('All plants in system, even with zero in stock:')
            for each_plant_id in Plants_dict:
                curr_plant_measurement, curr_plant_pack_or_pot, curr_plant_name, \
                curr_plant_size, curr_qty_in_stock = get_plant_info(each_plant_id)
                
                #display floats with .0 as integers #https://stackoverflow.com/a/36981428  #Shan Dou   feb 12 2017
                from decimal import Decimal
                curr_plant_size=f'{Decimal(curr_plant_size).normalize():f}'
                
                #plantcell not useful, but cm and litres are, this ensure this  information is shown
                #as it is on the current system
                if curr_plant_measurement == 'plantcell':
                    curr_plant_measurement = ''
                else:
                    #add contents of measurement to column 'size'
                    curr_plant_size=str(curr_plant_size)+' '+str(curr_plant_measurement)
                #put pack or pot word into the size column
                curr_plant_size=curr_plant_size+' '+curr_plant_pack_or_pot
                
                #get stock split infor for groups
                sunnyridge_total,greenheart_total,independents_total = display_stock_splitting_info(customer_id_input,each_plant_id, curr_week, curr_year)
                #get reserve quantities for dataframe
                reserve_quantity_allowed,reserve_stock_remaining,reserve_stock_taken_so_far=show_reserve_orders_info(each_plant_id,customer_id_input) #plant_id,customer_id
             
                plant_stock_data.append(
                {
                    'plant id': each_plant_id, 'curr plant name': curr_plant_name,
                    'curr plant size': curr_plant_size,
                    'curr qty in stock': curr_qty_in_stock,
                    'stock units rqd': stock_units_rqd,
                    'sunnyridge split': sunnyridge_total, 
                    'greenheart split': greenheart_total, 
                    'independent split': independents_total,
                    'reserve quantity':reserve_quantity_allowed, 
                    'reserve stock remaining':reserve_stock_remaining, 
                    'reserves stock taken so far':reserve_stock_taken_so_far
                })
               
        if user_choice == 2:
            print('Plants with above zero stock in system:')
            for each_plant_id in Plants_dict:
                
                curr_plant_measurement, curr_plant_pack_or_pot, curr_plant_name, curr_plant_size, \
                curr_qty_in_stock = get_plant_info(each_plant_id)
                
                #display floats with .0 as integers #https://stackoverflow.com/a/36981428  #Shan Dou   feb 12 2017
                from decimal import Decimal
                curr_plant_size=f'{Decimal(curr_plant_size).normalize():f}'
                
                #plantcell not useful, but cm and litres are, this ensure this  information is shown
                #as it is on the current system
                if curr_plant_measurement == 'plantcell':
                    curr_plant_measurement = ''
                else:
                    #add contents of measurement to column 'size'
                    curr_plant_size=str(curr_plant_size)+' '+str(curr_plant_measurement)
                #put pack or pot word into the size column
                curr_plant_size=curr_plant_size+' '+curr_plant_pack_or_pot
                
                #get stock split infor for groups
                sunnyridge_total,greenheart_total,independents_total = display_stock_splitting_info\
                (customer_id_input,each_plant_id, curr_week, curr_year)
                print('')
                #get reserve quantities for dataframe
                reserve_quantity_allowed,reserve_stock_remaining,reserve_stock_taken_so_far=show_reserve_orders_info\
                (each_plant_id,customer_id_input) #plant_id,customer_id

                if int(Plants_dict[each_plant_id]['qty_in_stock']) > 0: 
                    plant_stock_data.append(
                    {
                        'plant id': each_plant_id, 'curr plant name': curr_plant_name,
                        'curr plant size': curr_plant_size, 
                        'curr qty in stock': curr_qty_in_stock,
                        'stock units rqd': stock_units_rqd,
                        'sunnyridge split': sunnyridge_total, 
                        'greenheart split': greenheart_total, 
                        'independent split': independents_total,
                        'reserve quantity':reserve_quantity_allowed, 
                        'reserve stock remaining':reserve_stock_remaining, 
                        'reserves stock taken so far':reserve_stock_taken_so_far
                    })
        dframe = pd.DataFrame(plant_stock_data)
        
        
        #atempt: instead of above line
        #dframe = pd.DataFrame(plant_stock_data,index_col=0)
        
        
        #attempt:
        #make index same as first column.
        #dframe = dframe(index_col=0)
        #dframe.reset_index()
        #dframe.set_index(dframe.columns[0])
        #dframe.index.name='plant id'
        
        #dframe.set_index('plant id')
        #dframe.pop(dframe.columns[0])

        
        
        return (dframe) 

        
def get_plant_info(plant_id_user_choice):
    #returns plant information on the current entered plant id        
    curr_plant_name=Plants_dict[plant_id_user_choice]['plant_name']
    curr_plant_size=Plants_dict[plant_id_user_choice]['plant_size']
    curr_plant_measurement=Plants_dict[plant_id_user_choice]['measurement']
    curr_plant_pack_or_pot=Plants_dict[plant_id_user_choice]['pack_or_pot']
    curr_qty_in_stock = int(Plants_dict[plant_id_user_choice]['qty_in_stock'])
    return(curr_plant_measurement,curr_plant_pack_or_pot,curr_plant_name,curr_plant_size,curr_qty_in_stock)
 

def display_stock_splitting_info(customer_id,plant_id,this_week,this_year): #'1', '1', '2023' gives 2 records
    #displays stock splitting information for groups based on current entered plant id, and current week 
    #number and year
    sunnyridge_total=0
    greenheart_total=0
    independents_total=0
    #for this week year and plant id, which groups have stock splitting info?
    for stock_splitting_line_id in Stock_splitting_dict:
        group_id_on_this_line = int(Stock_splitting_dict[stock_splitting_line_id]['group_id'])
        #print('group_id_on_this_line currently:',group_id_on_this_line)
        #print('Customer_groups_members_dict[group_id_on_this_line][group_name] currently:',\
        #Customer_groups_members_dict[group_id_on_this_line]['group_name'])
        group_name_on_this_line = \
        Customer_groups_members_dict[group_id_on_this_line]['group_name']

        if (int(Stock_splitting_dict[stock_splitting_line_id]['plant_id']) == plant_id and
                    int(Stock_splitting_dict[stock_splitting_line_id]['week_no']) == this_week and
                    int(Stock_splitting_dict[stock_splitting_line_id]['year']) == this_year):
                if group_id_on_this_line == 1: #sunnyridge
                    sunnyridge_total=int(Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated'])
                elif group_id_on_this_line == 2: #greenheart 
                    greenheart_total=int(Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated'])
                elif group_id_on_this_line == 3: #'independents': 
                    independents_total=int(Stock_splitting_dict[stock_splitting_line_id]['qty_of_plant_allocated'])
    return (sunnyridge_total,greenheart_total,independents_total)
   
    
def show_reserve_orders_info(plant_id,customer_id):
    #As part of problem 3:
    #shows reserve order quantities
    #dict_for_df={}
    #no reserve order has the same combination of customer id and pLant id
    reserve_quantity_allowed=0
    reserve_stock_remaining=0
    reserve_stock_taken_so_far=0
    for reserve_order_id in Reserve_orders_dict:
        #make the values zero, these will be changed if the plant id and customer id are found

        if (int(Reserve_orders_dict[reserve_order_id]['plant_id']) == \
        plant_id and int(Reserve_orders_dict[reserve_order_id]['customer_id']) == customer_id):
            #save reserve quantities allowed and remaining and stock taken so far 
            reserve_quantity_allowed=Reserve_orders_dict[int(reserve_order_id)]['reserve_quantity_allowed']
            reserve_stock_remaining=Reserve_orders_dict[int(reserve_order_id)]['reserve_stock_remaining']
            reserve_stock_taken_so_far= reserve_quantity_allowed - reserve_stock_remaining
        #else:
        #    #else do nothing and for this unique combo of pant id and customer id no reserve order was found
        #    #so they will stay as zero
        #    dict_for_df[str(plant_id)+str(customer_id)]=reserve_quantity_allowed
        #    dict_for_df[str(plant_id)+str(customer_id)]=reserve_stock_remaining
        #    dict_for_df[str(plant_id)+str(customer_id)]=reserve_stock_taken_so_far
        
    return (reserve_quantity_allowed,reserve_stock_remaining,reserve_stock_taken_so_far)

                  

def generate_unique_id(curr_dictionary):
    #generates unique ids for use as primary keys in dictionaries
    #generate unique id for any dictionary automatically so we can add to it with next logical id number
    no_dictionary_ids_yet = (len(curr_dictionary) == 0)
    if no_dictionary_ids_yet:
        new_dictionary_id = 1 
    else:
        new_dictionary_id = len(curr_dictionary)+1    
    return (new_dictionary_id)


def display_customers():
    #displays a list of all customers in the database
    print('List of customers:')
    df = pd.DataFrame(Customers_dict).T
    #print(df.head())
    df.index.name='customer_id'
    display(df)

    
    
def highlight_late(recent_dataframe):
    #returns coloured rows based on whether plant ids were allocated or not
    if recent_dataframe['allocated'] == 'Yes':
        return ['background-color: green' for row in recent_dataframe]
    elif recent_dataframe['allocated'] == 'No':
        return ['background-color: red' for row in recent_dataframe]
    else: 
        return ['background-color: white' for row in recent_dataframe]
    

def main_display_table(plant_id,curr_week,curr_year,group_id_to_update,customer_id_input):
    #displays list of plant ids and whether they wre allocated or not
    for plant_id in Plants_dict:
        plant_id=int(plant_id)
        #fill new dict with Plants updated info
        #display floats with .0 as integers #https://stackoverflow.com/a/36981428  #Shan Dou   feb 12 2017
        size_from_dict=Plants_dict[plant_id]['plant_size']
        #sort out plant size into one field from concatenation of others
        from decimal import Decimal
        curr_plant_size=f'{Decimal(size_from_dict).normalize():f}'
        #plantcell not useful, but cm and litres are, this ensure this  information is shown
        #as it is on the current system
        new_temp_measurement=Plants_dict[plant_id]['measurement']
        if Plants_dict[plant_id]['measurement'] == 'plantcell':
            new_temp_measurement = ''
        else:
            #add contents of measurement to column 'size'
            curr_plant_size=str(size_from_dict)+' '+new_temp_measurement
        #put pack or pot word into the size column
        concat_size=curr_plant_size + ' ' + Plants_dict[plant_id]['pack_or_pot']
        updated_everything_dict[int(plant_id)] = {}
        updated_everything_dict[int(plant_id)]['plant id']=plant_id
        updated_everything_dict[int(plant_id)]['plant name']=Plants_dict[plant_id]['plant_name']
        updated_everything_dict[int(plant_id)]['plant size']= concat_size
        updated_everything_dict[int(plant_id)]['quantity in stock']=Plants_dict[plant_id]['qty_in_stock']

        #fill new dict with stock splitting updated info
        #reset to keep as zero unless over written
        updated_everything_dict[int(plant_id)]['sunnyridge split']=0
        updated_everything_dict[int(plant_id)]['greenheart split']=0
        updated_everything_dict[int(plant_id)]['independent split']=0
        for stock_splitting_line_id in Stock_splitting_dict:   
            if (int(Stock_splitting_dict[int(stock_splitting_line_id)]['plant_id']) == plant_id 
                    and int(Stock_splitting_dict[int(stock_splitting_line_id)]['week_no']) == curr_week
                    and int(Stock_splitting_dict[int(stock_splitting_line_id)]['year']) == curr_year
                    and int(Stock_splitting_dict[int(stock_splitting_line_id)]['group_id']) == group_id_to_update):

                if group_id_to_update == 1: #sunnyridge
                    updated_everything_dict[int(plant_id)]\
                    ['sunnyridge split']= Stock_splitting_dict[int(stock_splitting_line_id)]['qty_of_plant_allocated']  # sunnyridge_split
                if group_id_to_update == 2: #greenheart 
                    updated_everything_dict[int(plant_id)]\
                    ['greenheart split']= Stock_splitting_dict[int(stock_splitting_line_id)]['qty_of_plant_allocated'] #greenheart_split
                if group_id_to_update == 3: #'independents': 
                    updated_everything_dict[int(plant_id)]\
                    ['independent split']= Stock_splitting_dict[int(stock_splitting_line_id)]['qty_of_plant_allocated'] #independent_split

        #fill new dict with reserve order updated info
        for reserve_order_id in Reserve_orders_dict:   
            if int(Reserve_orders_dict[int(reserve_order_id)]['customer_id'])==customer_id_input \
            and int(Reserve_orders_dict[int(reserve_order_id)]['plant_id'])==plant_id:
                #make dframe cols of (rsvQuantAllwdPerCust.reserve_stock_remaining,reserve_stock_taken_so_far) ELSE put zero for dframe cols of rsvQuantAllwdPerCust.reserve_stock_remaining,reserve_stock_taken_so_far

                updated_everything_dict[int(plant_id)]['reserve quantity']=\
                Reserve_orders_dict[int(reserve_order_id)]['reserve_quantity_allowed']
                updated_everything_dict[int(plant_id)]['reserve stock remaining']=\
                Reserve_orders_dict[int(reserve_order_id)]['reserve_stock_remaining']
                updated_everything_dict[int(plant_id)]['reserve stock taken so far']=\
                Reserve_orders_dict[int(reserve_order_id)]['reserve_quantity_allowed']-\
                Reserve_orders_dict[int(reserve_order_id)]['reserve_stock_remaining']


    #make up to date dataframe of all changes fresh from dictionary all rippled down            
    updated_dframe=pd.DataFrame.from_dict(updated_everything_dict).T
    updated_dframe['allocated'] = updated_dframe['plant id'].map(plant_id_allocated)
    updated_dframe['stock_rqd'] = updated_dframe['plant id'].map(plant_id_stock)
    #updated_dframe['stock_rqd']=updated_dframe['stock_rqd'].round().astype(int)

    #to get the column into integers after being mapped from a dictionary
    #https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int/50132405#50132405
    updated_dframe['stock_rqd'] = updated_dframe['stock_rqd'].fillna(-1)
    updated_dframe['stock_rqd'] = updated_dframe['stock_rqd'].astype(int)
    updated_dframe['stock_rqd'] = updated_dframe['stock_rqd'].astype(str)
    updated_dframe['stock_rqd'] = updated_dframe['stock_rqd'].replace('-1', np.nan)

    #save dframe in more ordered way visualy
    updated_dframe_ordered = \
    updated_dframe[['plant id','plant name','plant size','quantity in stock',\
                    'stock_rqd', 'sunnyridge split', 'greenheart split', 'independent split',
                    'reserve quantity', 'reserve stock remaining', \
                    'reserve stock taken so far', 'allocated']]
    updated_dframe_ordered['allocated'] = \
    np.where((updated_dframe_ordered['allocated'] != 'Yes') \
             & (updated_dframe_ordered['allocated'] != 'No'), \
             'Not visited', updated_dframe_ordered['allocated'])
    updated_dframe_ordered=updated_dframe_ordered.fillna(0)
    
    return(updated_dframe_ordered)
    


main in which there are dictionaries at the start.


In [71]:
def main():  
    #so we can save the acknowledgement order to a csv file and display it as a dataframe with a headed index column 
    ack_order_data = []
    
    #so we can display the latest updates after order details are entered to th e screen
    #updated_stock_data = [] 
    
    #set to zero for initializing it
    stock_units_rqd = 0   
    today = datetime.today()
    #convert these to int as we are passing them to functions
    #rule is all number variables passed to functions are ints as we 
    # do operations on them in the functions, then we save back to str when saving to dictionary
    curr_week = int(today.isocalendar()[1])
    curr_year=int(datetime.now().year)
    curr_month=int(datetime.now().month)
    curr_day=int(datetime.now().day)

    #GILLS ADDITION: DISPLAY ALL CUSTOMERS HERE
    display_customers()
    
    #all IDS are INT
    #CHOOSE CUSTOMER ID AND PUT IN YOUR SALES TEAM ID
    customer_id_input=int(input('enter customerID:'))
    sales_team_id_input=int(input('enter salesTeamID:'))
    
    
    #save customer group id and name for this customer id for use later:
    group_id_to_update=int(Customers_dict[customer_id_input]['customer_group_id'])
    group_name_to_update = (Customer_groups_members_dict[group_id_to_update]['group_name'])
    
    #get user input about what plant ids to display
    stock_display_user_choice=int(input("Choose: 1 See all stock lines in database, 2: See show only \
    stock lines with stock available"))

    
    #new plant object so we can display plantIDs with stock above zero and info within loop
    new_plants1=Plants()
    
    #display all stock or stock above zero depending on user choice
    dframe_all_plants=new_plants1.display_plant_info \
    (stock_display_user_choice, customer_id_input, curr_week, curr_year, stock_units_rqd)
    from IPython.display import display
    
    
    #attempt
    #dframe_all_plants.index.name='bobe'
    dframe_all_plants.index = dframe_all_plants['plant id']
    dframe_all_plants.pop(dframe_all_plants.columns[0])
    
    #print('first  with index  being bobe')
    display(dframe_all_plants)
    
    
    #dframe_all_plants.index.name='plant_id'
    #print('now with plant id')
    #display(dframe_all_plants)
    
    
    
    #keeps updates of everything that is amended after all calcs
    updated_everything_dict={}
    
    #display(dframe_all_plants)
    carry_on = '1'
    stock_line_number=1
    grand_total=0
    
    #set this as no so that only those rows allocated we can set as yes, then display yes  
    #as green rows, no as red later
    dframe_all_plants['allocated']='Not attempted'
    set_of_failed_updates=set()
    #get next id number this order will have
    this_order_id=int(generate_unique_id(Plant_orders_dict))
    
    allocated_attempted=False
    all_green_rows =False #meaning after allocation attempt, no problems occured
    while carry_on == '1' and all_green_rows == False: #meaning add another order line
        #keep track of which did not update due to lack of stock
        
        #if you have tried to allocated but got red rows
        if allocated_attempted==True:
            #show recent table of plants and their numbers 
            #nb.we will have overwritten the updated one each time through loop
            display(updated_dframe_ordered.style.apply(highlight_late, axis=1))
            
            #format the set of failed update numbers to display without curly brackets and display message
            newstring=''
            for each in set_of_failed_updates:
                newstring=newstring+str(each)+','
            #display message to help user
            if len(set_of_failed_updates) > 1:
                print(f'Acknowledgement not possible. Cannot allocate plant ids:',newstring[:-1],'. Try new quantities...')
            else:
                print(f'Acknowledgement not possible. Cannot allocate plant id:',newstring[:-1],'. Try a new quantity...')
        
        #get plant id
        #WE HAVE TO USE dframe_all_plants AT TOP OF WHILE LOOP NOTHING ELSE!
        plant_id_user_choice=int(input('Enter plant id for this line of the order:'))

        old_stock_line_number = stock_line_number
        #GO TO THIS FUNCTION TO ENTER YOUR REQUIRED QUANTITY FOR ONE ORDER LINE
        #AND GET A BUNCH OF VARIABLES BACK WITH INFO ON THE PLANT AND THEORETICAL 
        #NEW STOCK LVL
        curr_plant_name, curr_plant_size, curr_plant_measurement, curr_plant_pack_or_pot, \
        curr_qty_in_stock, plant_barcode, plant_price, \
        vat_for_one_plant, order_line_total, sold_in_multiples_of, \
        stock_units_rqd, grand_total, stock_line_number, \
        total_units_to_remove_from_stock, set_of_failed_updates, \
        theoretical_new_stock_value,dframe_all_plants = add_ord_pt1_enter_quant_calc_totals\
                                                        (this_order_id, plant_id_user_choice,\
                                                        stock_line_number, grand_total, \
                                                        dframe_all_plants,set_of_failed_updates) 
        new_stock_line_number = stock_line_number
        
        #--------------------about displaying a table---------------------------
        if allocated_attempted==False:
            #print('------here is plant_id_stock at this point:---------')
            #print('should be empty:',plant_id_stock)
            #the stock units required part is done in another function 
            #called add_ord_pt1_enter_quant_calc_totals 
            
            #show dataframe at the start with up to date dictionaries pre allocation attempt
            
            
            
            
            #WAITWAIT
            
            #attempt:
            dframe_all_plants_pre_allocation=main_display_table\
            (plant_id_user_choice,curr_week,curr_year,group_id_to_update,customer_id_input)
            
            
            #attempt2:
            #dframe_all_plants_pre_allocation.reset_index()
            #dframe_all_plants_pre_allocation.set_index(dframe_all_plants_pre_allocation.columns[0])
            #dframe_all_plants_pre_allocation.index.name='plant id'
            #dframe_all_plants_pre_allocation.pop(dframe_all_plants_pre_allocation.columns[0])
    
            #attempt
            dframe_all_plants_pre_allocation.index.name='plant_id'
            dframe_all_plants_pre_allocation.pop(dframe_all_plants_pre_allocation.columns[0])
            
            
            
            
            display(dframe_all_plants_pre_allocation.iloc[: , :-1]) # we dont want to 
                                                                    # see last 'allocated' 
                                                                    # column at this point
        #only here when red lines and have tried to allocate
        else:
            updated_dframe_ordered.loc[plant_id_user_choice,'stock_units_rqd'] = \
            stock_units_rqd
            #change order of columns
            updated_dframe_ordered.iloc[:, [0,1,2,3,10,5,6,7,8,9]]
            #make the floats into integers in this column integers
            updated_dframe_ordered.stock_rqd = \
            updated_dframe_ordered['stock_units_rqd'] = \
            updated_dframe_ordered['stock_units_rqd'].astype('Int64')
            display(updated_dframe_ordered.style.apply(highlight_late, axis=1))
        #-----------------------end of displaying a table------------------------                                       
          
            
        #------save all info to dictionaries regarding reserve orders, stock splitting and plant stock levels-----
        #IF NO PROBLEMS WITH PLANTS STOCK GOING BELOW ZERO:
        if old_stock_line_number != new_stock_line_number \
                and total_units_to_remove_from_stock != 0:
            #print(f'Success .. allocating for customer id {customer_id_input}, week: {curr_week}')
            # make a dictionary for a DataFrame 
            
            #display floats with .0 as integers #https://stackoverflow.com/a/36981428  #Shan Dou feb 12 2017
            from decimal import Decimal
            curr_plant_size=f'{Decimal(curr_plant_size).normalize():f}'

            #plantcell not useful, but cm and litres are, this ensure this  information is shown
            #as it is on the current system
            if curr_plant_measurement == 'plantcell':
                curr_plant_measurement = ''
            else:
                #add contents of measurement to column 'size'
                curr_plant_size=str(curr_plant_size)+' '+str(curr_plant_measurement)
            
            #put pack or pot word into the size column
            curr_plant_size=curr_plant_size+' '+curr_plant_pack_or_pot
            
            ack_order_data.append({'order line number':stock_line_number-1,
                                   'plant id':plant_id_user_choice,
                                   'name':curr_plant_name,
                                   'size':curr_plant_size,
                                   'qty':stock_units_rqd,
                                   'Sold in multiples of':sold_in_multiples_of,
                                   'units':total_units_to_remove_from_stock,
                                   'barcode':plant_barcode,
                                   'price':plant_price,
                                   'vat':vat_for_one_plant,
                                   'orderline total':order_line_total,
                                   })

            
            #call function pt2 to change Plants_dict stock,
            this_plant_id_new_qty_in_stock = add_ord_pt_2_update_plant_id_stock_lvls\
                                            (plant_id_user_choice,total_units_to_remove_from_stock)
        

            #update stock splitting dictionary levels, 
            #and save results to variables to update stock splitting stock levels
            independent_split,sunnyridge_split,\
            greenheart_split=add_ord_pt3_update_stock_splitting_stock_lvls\
            (curr_week,curr_year, plant_id_user_choice,\
            total_units_to_remove_from_stock,group_id_to_update)
            
            
            #change Reserve orders dictionary levels
            #get vars to update reserve order stock levels
            reserve_quantity,\
            reserve_stock_remaining,\
            reserve_stock_taken_so_far = add_ord_pt4_update_rsv_ord_stock_lvls \
            (plant_id_user_choice, customer_id_input, total_units_to_remove_from_stock)

            
            #update reserve orders dict with updates reserve stock remaining
            for each_reserve_order_id in Reserve_orders_dict:
                if int(Reserve_orders_dict[each_reserve_order_id]['customer_id']) == customer_id_input \
                        and int(Reserve_orders_dict[each_reserve_order_id]['plant_id']) == plant_id_user_choice:
                    #print('found a match, am updating stock remainin in reserve dict')
                    Reserve_orders_dict[each_reserve_order_id]['reserve_stock_remaining']=\
                    reserve_stock_remaining
        
            #update stock splitting dict with updated variables
            #for this week year and plant id, which groups have stock splitting info?
            for stock_splitting_line_id in Stock_splitting_dict:
                group_id_on_this_line = int(Stock_splitting_dict[stock_splitting_line_id]\
                                                                             ['group_id'])
                group_name_on_this_line = Customer_groups_members_dict[group_id_on_this_line]\
                                                                               ['group_name']

                if (int(Stock_splitting_dict[stock_splitting_line_id]\
                        ['plant_id']) == plant_id_user_choice and\
                        int(Stock_splitting_dict[stock_splitting_line_id]\
                        ['week_no']) == curr_week and\
                        int(Stock_splitting_dict[stock_splitting_line_id]['year']) == curr_year):
                    if group_id_on_this_line == 1: #sunnyridge
                        Stock_splitting_dict[int(stock_splitting_line_id)]\
                        ['qty_of_plant_allocated']= sunnyridge_split
                    if group_id_on_this_line == 2: #greenheart 
                        Stock_splitting_dict[int(stock_splitting_line_id)]\
                        ['qty_of_plant_allocated']= greenheart_split
                    if group_id_on_this_line == 3: #'independents': 
                        Stock_splitting_dict[int(stock_splitting_line_id)]\
                        ['qty_of_plant_allocated']= independent_split
            
            #update dataframe from dictionary
            updated_dframe_all_plants=new_plants1.display_plant_info \
            (1, customer_id_input, curr_week, curr_year, stock_units_rqd)
            stock_split_cols = pd.DataFrame(Stock_splitting_dict)
            res_cols = pd.DataFrame(Reserve_orders_dict)
        #------end of saving info to dictionaries regarding reserve orders, stock splitting and plant stock levels-----
        
        
        
        #EITHER SAVED OR NOT, AND HAS RIPPLED DOWN IF NEEDED AT THIS POINT
        #to Plants_dict stock,stock_splitting_dict lvls, Reserve_orders_dict
        print('1 : add another line')
        print('2 : allocate stock to the order')
        carry_on=input('Enter your choice: ')
        
        
        #------set flags to continue the while loop if there were problems,  and tell user number of failed----
        #------updates if there were red rows------------------------------------------------------------------
        if carry_on == '2':# and set_of_failed_updates:
            allocated_attempted=True #set flag so we display updated dataframe from now on
            if set_of_failed_updates:
                print (f'Problem: {len(set_of_failed_updates)} rows did not allocate.')         
            else:
                print (f'All rows have been allocated.')         
            
            #set flag to leave loop when there are no failed updated
            if len(set_of_failed_updates) == 0:
                all_green_rows=True
            else:
                print(f'Problem: Plant ids {set_of_failed_updates} in red were not allocated.')
            
            #this forces the while loop to continue, since there are red rows
            carry_on='1'
            
            #save updated dictionaries to dataframe 
            updated_dframe_ordered=main_display_table(plant_id_user_choice,curr_week,curr_year,\
                                                      group_id_to_update,customer_id_input)
        #---end of telling user info on the updates and setting flags ------------
    
    
    #---- Here when no red rows. Now it's time to choose to acknowlege or not-----
    #---- and to choose to save permanently or not--------------------------------
    #setup some variables to help format the acknowledgement if they choose to do it
    time_now = datetime.now()
    current_time = time_now.strftime("%H:%M:%S")
    date_and_time=str(datetime.now().strftime('%d-%m-%Y'))+' '+str(current_time)
    plant_order_id=int(this_order_id)

    #First we save the order to a dictionary within the Plant orders dictionary 
    Plant_orders_dict[plant_order_id] = {}
    Plant_orders_dict[plant_order_id]['customer_id'] = customer_id_input
    Plant_orders_dict[plant_order_id]['sales_team_id'] = sales_team_id_input
    Plant_orders_dict[plant_order_id]['date_order_ack_sent'] = date_and_time 
    Plant_orders_dict[plant_order_id]['grand_total'] = grand_total
    
    #show the final updated all green rows dataframe
    from IPython.display import display
    
    
    #attempt:
    updated_dframe_ordered.index.name='plant_id'
    updated_dframe_ordered.pop(updated_dframe_ordered.columns[0])
    display(updated_dframe_ordered.style.apply(highlight_late, axis=1))

    #ask user if they want to acknowledge order or not
    print('There are no failed updates, you may acknowledge this order if required.')
    ack_choice=input('Would you like to Acknowledge this order? y/n: ')         
    if ack_choice == 'n':  
        Plant_orders_dict[this_order_id]['ack_sent'] = 'n'
    if ack_choice == 'y':
        #display the order acknowledgement on the screen as it will appear to the customer in an email
        #final version this would trigger an email too of the order acknowledgement
        Plant_orders_dict[this_order_id]['ack_sent'] = 'y'
        #round grand total to get rid of too many decimal places
        grand_total_rounded = round(grand_total, 2)
        current_date_and_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print('')
        print('')
        print('Angels plant order Acknowledgement')
        print('')
        print('Angels Plants                                       ',\
              Customers_dict[customer_id_input]['customer_first_name'])
        print('Stroud                                              ',\
              Customers_dict[customer_id_input]['customer_last_name'])   
        print('Gloucestershire GL5 8QL                             ',\
              Customers_dict[customer_id_input]['customer_address'])
        print('                                               Tel.:',\
              Customers_dict[customer_id_input]['customer_tel_no'])
        print('')
        #print('Date order Acknowledged: ',curr_day, '/', curr_month, '/', curr_year)
        print('Order ID: ',this_order_id)
        print('Date and time order acknowledged:',current_date_and_time)
        print('')
        ack_order_dframe = pd.DataFrame(ack_order_data)

        from IPython.display import display
        display(ack_order_dframe)
        print('')      
        print('                                                          \
                      grand_total:',grand_total_rounded) 


    #finaly let user if they want to save changes to database permanently 
    #ie. save the dictionaries full of this order info, to csv files
    save_choice = input('Save changes permanently? y/n: ')
    if save_choice != 'y': #if n or any other letter apart from y is entered  
        print('Any changes from this order have NOT been saved.')
    else:
        #user has chosen to save changes permanently
        #make dataframes from dictionary, and save to csv, even if they do not exist
        df1 = pd.DataFrame(Customer_groups_members_dict).T
        df1.to_csv('sql_data/Customer_groups_members_dict.csv', mode='w', index = 1, index_label='Group id')

        df2 = pd.DataFrame(Reserve_orders_dict).T
        df2.to_csv('sql_data/Reserve_orders_dict.csv', mode='w', index = 1, index_label='Reserve order id')

        df3 = pd.DataFrame(Nursery_managers_dict).T
        df3.to_csv('sql_data/Nursery_managers_dict.csv', mode='w', index = 1, index_label='Manager id')

        df4 = pd.DataFrame(Order_lines_dict).T
        df4.to_csv('sql_data/Order_lines_dict.csv', mode='w', index = 1, index_label='Order line id')

        df5 = pd.DataFrame(Sales_team_members_dict).T
        df5.to_csv('sql_data/Sales_team_members_dict.csv', mode='w', index = 1, index_label='Sales team id')

        df6 = pd.DataFrame(Plant_orders_dict).T
        df6.to_csv('sql_data/Plant_orders_dict.csv', mode='w', index = 1, index_label='Plant order id')

        df7 = pd.DataFrame(Customers_dict).T
        df7.to_csv('sql_data/Customers_dict.csv', mode='w', index = 1, index_label='Customer id')

        df8 = pd.DataFrame(Stock_splitting_dict).T
        df8.to_csv('sql_data/Stock_splitting_dict.csv', mode='w', index = 1, index_label='Stock line id')

        df9 = pd.DataFrame(Plants_dict).T
        df9.to_csv('sql_data/Plants_dict.csv', mode='w', index = 1, index_label='Plant id')

        #update excel spreadsheet to give me test data
        writer = pd.ExcelWriter('excel_data/Testing Data.xlsx', engine = 'xlsxwriter')
        df1.to_excel(writer, sheet_name = 'Customer groups', index = True, index_label = 'Group_id')
        df2.to_excel(writer, sheet_name = 'Reserve orders', index = True, index_label = 'Reserve_order_id')
        df3.to_excel(writer, sheet_name = 'Nursery managers', index = True, index_label = 'Manager_id')
        df4.to_excel(writer, sheet_name = 'Order lines', index = True, index_label = 'Order_line_id')
        df5.to_excel(writer, sheet_name = 'Sales team members', index = True, index_label = 'Sales_team_id')
        df6.to_excel(writer, sheet_name = 'Plant orders', index = True, index_label = 'Plant_order_id')
        df7.to_excel(writer, sheet_name = 'Customer', index = True, index_label = 'Customer_id')
        df8.to_excel(writer, sheet_name = 'Stock Splitting', index = True, index_label = 'Stock_line_id')
        df9.to_excel(writer, sheet_name = 'Plants', index = True, index_label = 'Plant_id')

        writer.save()
        print('All changes have been saved permanently, thank you for placing an order.')
        
        
#Execute main() function, useful to run easily from computer
if __name__ == '__main__':
    main()


List of customers:


,customer_group_id,customer_first_name,customer_last_name,customer_address,customer_tel_no
customer_id,,,,,
1,1,Simon,Sunday,"Sunnyridge Branch 5, Sunnyridge lane, London, UK",0900 232 232
2,1,Susan,Sussex,"Sunnyridge Branch 2, Sunnyridge lane, London, UK",02300 442 241
4,2,Tate,Greenway,"Greenheart Branch 1, Woking, Surrey",01180 431 111
5,3,Tom,Innes,"Innes garden centre, Stroud, Gloucestershire",01453 479 238
6,3,Bob,India,"India garden centre, Slade, Reading",01179 439 238


enter customerID:1
enter salesTeamID:1
Choose: 1 See all stock lines in database, 2: See show only     stock lines with stock available1

All plants in system, even with zero in stock:


,curr plant name,curr plant size,curr qty in stock,stock units rqd,sunnyridge split,greenheart split,independent split,reserve quantity,reserve stock remaining,reserves stock taken so far
plant id,,,,,,,,,,
1,Pansy White,10.5 cm pot,915,0,915,0,0,210,135,75
3,Geranium Mix,20 pack,0,0,0,0,0,0,0,0
4,Marigold French Yellow,9 cm pot,828,0,0,0,0,0,0,0
5,Dahlia Coral Flame,2 litre pot,100,0,0,0,0,0,0,0
6,Salvia Blue,5 litre pot,0,0,0,0,0,0,0,0


Enter plant id for this line of the order:1
Enter quantity required:1


...

1 : add another line
2 : allocate stock to the order
Enter your choice: 2
All rows have been allocated.


,plant name,plant size,quantity in stock,stock_rqd,sunnyridge split,greenheart split,independent split,reserve quantity,reserve stock remaining,reserve stock taken so far,allocated
plant_id,,,,,,,,,,,
1,Pansy White,10.5 cm pot,900,1,900,0,0,210,120,90,Yes
3,Geranium Mix,20 pack,0,0,0,0,0,0,0,0,Yes
4,Marigold French Yellow,9.0 cm pot,756,4,0,0,0,0,0,0,Yes
5,Dahlia Coral Flame,2.0 litre pot,100,0,0,0,0,0,0,0,Yes
6,Salvia Blue,5.0 litre pot,0,0,0,0,0,0,0,0,Not visited


There are no failed updates, you may acknowledge this order if required.
Would you like to Acknowledge this order? y/n: y


Angels plant order Acknowledgement

Angels Plants                                        Simon
Stroud                                               Sunday
Gloucestershire GL5 8QL                              Sunnyridge Branch 5, Sunnyridge lane, London, UK
                                               Tel.: 0900 232 232

Order ID:  9
Date and time order acknowledged: 2023-09-05 23:23:38



,order line number,plant id,name,size,qty,Sold in multiples of,units,barcode,price,vat,orderline total
0,1,1,Pansy White,10.5 cm pot,1,15,15,7231231312391,0.80,0.16,14.4
1,2,4,Marigold French Yellow,9 cm pot,4,18,72,1211111111124,0.75,0.15,64.8



                                                                                grand_total: 79.2
Save changes permanently? y/n: y
All changes have been saved permanently, thank you for placing an order.


# 
